In [20]:
import os
import cv2
import torch
import numpy as np
from torchvision import transforms
from torchvision import models
import torch.nn as nn

class DeepFakeDetector(nn.Module):
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        self.resnext = models.resnext50_32x4d(weights='ResNeXt50_32X4D_Weights.IMAGENET1K_V1')
        self.resnext = nn.Sequential(*list(self.resnext.children())[:-2])
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.lstm = nn.LSTM(2048, 2048, batch_first=True)
        self.linear = nn.Linear(2048, 1)  # Output 1 for binary classification

    def forward(self, x):
        batch_size, num_frames, channels, height, width = x.size()
        x = x.view(batch_size * num_frames, channels, height, width)
        features = self.resnext(x)
        features = self.adaptive_pool(features)
        features = torch.flatten(features, start_dim=1)
        features = features.view(batch_size, num_frames, -1)
        lstm_output, _ = self.lstm(features)
        lstm_output = lstm_output[:, -1, :]
        output = self.linear(lstm_output)
        return output  # Return logits directly

# Load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepFakeDetector()  # Initialize the model class
model.load_state_dict(torch.load(r'C:\deepfake videos\models\dfd_model_weights.pth', map_location=device))  # Load weights
model.to(device)
model.eval()

# Define the transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((112, 112)),  # Reduced size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def preprocess_video(video_path, num_frames=100):
    """
    Preprocesses the video by loading pre-processed frames.
    """
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    # Extract frames from the video file
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()

    # Check if there are enough frames
    if len(frames) == 0:
        print(f"Warning: No frames found in {video_path}.")
        return None

    # Select frames to match the desired number
    if len(frames) >= num_frames:
        frame_indices = np.linspace(0, len(frames) - 1, num_frames).astype(int)
        selected_frames = [frames[i] for i in frame_indices]
    else:
        selected_frames = frames + [frames[-1]] * (num_frames - len(frames))

    # Apply transformations
    transformed_frames = [transform(frame) for frame in selected_frames]
    return torch.stack(transformed_frames)

def predict_video(video_path, threshold=0.35):
    """
    Predicts whether the given video is real or fake.
    """
    video_tensor = preprocess_video(video_path)
    if video_tensor is None:
        return

    video_tensor = video_tensor.unsqueeze(0).to(device)  # Add batch dimension
    with torch.no_grad():
        outputs = model(video_tensor)
        outputs = torch.sigmoid(outputs)
        prediction = (outputs >= threshold).item()  # Get prediction as 0 or 1

    label = "Real" if prediction == 0 else "Fake"
    print(f"Prediction for {video_path}: {label}")

# Main function to take video input
if __name__ == "__main__":
    video_path = input("Enter the path to the video file: ")
    predict_video(video_path)


C:\Users\Atharva\AppData\Local\Temp\ipykernel_22704\4242929494.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r'C:\deepfake videos\mod

Prediction for C:\deepfake videos\FF_Face_only_data\000_003.mp4: Fake


In [23]:
import cv2

def play_video(video_path):
    # Create a VideoCapture object to read the video
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return

    while True:
        # Read a frame from the video
        ret, frame = cap.read()

        if not ret:
            break  # Exit the loop if the video has ended

        # Display the frame in a window named 'Video'
        cv2.imshow('Video', frame)

        # Wait for 25 ms before moving on to the next frame
        # Press 'q' to quit the video early
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    # Release the video capture object and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

# Path to the video
video_path = r'C:\deepfake videos\FF_Face_only_data\962.mp4'

# Play the video
play_video(video_path)
